In [ ]:
spark

# RDD creation

In [ ]:
r = range(0,10)

In [ ]:
rdd1 = sc.parallelize(r)
rdd1

In [ ]:
rdd1.collect()

In [ ]:
rdd1.first()

In [ ]:
rdd1.take(5)

In [ ]:
rdd1.count()

In [ ]:
rdd_text = sc.textFile('LICENSE_spark.txt')

In [ ]:
rdd_text.take(5)

In [ ]:
rdd_text.count()

# Operations on RDD

In [ ]:
rdd1.filter(lambda x: x%2 == 0)

In [ ]:
rdd1.filter(lambda x: x%2 == 0).collect()

In [ ]:
rdd1.map(lambda x: 2*x).collect()

In [ ]:
rdd1.map(lambda x: 2*x).sum()

In [ ]:
rdd1.map(lambda x: [y for y in  range(1,x)]).collect()

In [ ]:
rdd1.flatMap(lambda x: [y for y in  range(1,x)]).collect()

In [ ]:
rdd2 = sc.parallelize(range(4,13))
rdd1.intersection(rdd2).collect()

In [ ]:
rdd2 = sc.parallelize(range(8,12))
rdd1.union(rdd2).collect()

In [ ]:
x = sc.parallelize([1,2,3])
y = sc.parallelize([10,11,12])
z = x.cartesian(y)
z.collect()

## Aggregations

In [ ]:
rdd1.reduce(lambda x,y: x + y)

In [ ]:
rdd1.aggregate(0, lambda x,y: x + y, lambda x,y: x + y)

In [ ]:
rdd1.aggregate(0, lambda x,y: max(x,y), lambda x,y: x + y)

In [ ]:
a = sc.parallelize(["negro", "azul", "blanco", "verde", "gris"], 2)

In [ ]:
a.aggregate(0, lambda x,y: x + len(y), lambda x,y: x + y)

## With keys

In [ ]:
z.keys().collect()

In [ ]:
a = sc.parallelize(["negro", "azul", "blanco", "verde", "gris"], 2)
b = a.groupBy(lambda x: len(x))
sorted([(x,sorted(y)) for (x,y) in b.collect()])

In [ ]:
b.countByKey()

In [ ]:
b.mapValues(len).collect()

In [ ]:
a = sc.parallelize(["azul", "verde", "naranja"], 3)
b = a.keyBy(lambda x: len(x))
c = sc.parallelize(["negro", "blanco", "gris"], 3)
d = c.keyBy(lambda x: len(x))
b.join(d).collect()

In [ ]:
b.leftOuterJoin(d).collect()

In [ ]:
b.rightOuterJoin(d).collect()

In [ ]:
b.fullOuterJoin(d).collect()

In [ ]:
a = sc.parallelize(["negro", "azul", "blaco", "verde", "gris"], 2)
b = a.map(lambda x: (len(x), x))
b.reduceByKey(lambda x,y: x + y).collect()

## Words count

In [ ]:
import re
pattern = re.compile('[^[^a-zA-Z ]')

In [ ]:
rdd_text = sc.textFile('LICENSE_spark.txt')

In [ ]:
rdd_text.take(5)

**Q1:** limpiar la lista, quitar espacios y caracteres non alfabeticos

**Q2:** separar las palabras

**Q3:** cuantas palabras differentes ?

**Q4:** palabras mas frequentes ?

# Accumulator

In [ ]:
acc = sc.accumulator(0)

In [ ]:
def initiate(x):
    acc.add(1)
    return (x,1)

In [ ]:
words_pairs2 = words.map(initiate)

In [ ]:
words_pairs2.foldByKey(0, lambda x,y: x+y).take(5)

In [ ]:
acc.value

In [ ]:
words_pairs2.foreach(lambda x: acc.value)

In [ ]:
list_nt = sc.parallelize(range(1,100))

In [ ]:
list_nt.foreach(lambda x: acc.add(1))

In [ ]:
acc.value

# Broadcast

In [ ]:
brands = SparkContext.broadcast(sc,["Apache", "Spark", "Hive", "AWS"])

In [ ]:
words_pairs3 = words\
    .filter(lambda x: x in brands.value)\
    .map(initiate)

In [ ]:
words_pairs3.foldByKey(0, lambda x,y: x+y).collect()

In [ ]:
br_ints = sc.broadcast([3,5])

In [ ]:
list_nt.filter(lambda x: all([x % i == 0 for i in br_ints.value])).collect()

# Analysis on numerical RDDs

In [ ]:
import random

In [ ]:
rdd2 = sc.parallelize([random.randint(0,10) for _ in range(25)])

In [ ]:
rdd2.stats()

In [ ]:
rdd2.histogram(5)

# Partitions

In [ ]:
rdd2.getNumPartitions()

In [ ]:
def printList(l): print('['+ ','.join([str(i) for i in l]) + ']')
rdd.foreachPartition(printList) # actually printing on stout of the 

In [ ]:
#def collectList(iterator): return list(iterator)
def collectList(iterator): yield list(iterator)
#rdd.mapPartitions(lambda i: list(i)).take(3)
rdd2.mapPartitions(collectList).collect()

In [ ]:
# there is a predefined function to do that: glom()
rdd2.glom().collect()

In [ ]:
def lenIter(iterator): yield sum(1 for x in iterator)
#def lenIter(iterator): return sum(1 for x in iterator)
rdd2.mapPartitions(lenIter).collect()

In [ ]:
print(str(rdd2.coalesce(2).glom().toDebugString()).replace('\\n','\n'))
rdd2.coalesce(2).glom().collect()

In [ ]:
rdd2.coalesce(6).glom().collect()

In [ ]:
print(str(rdd2.repartition(2).glom().toDebugString()).replace('\\n','\n'))
rdd2.repartition(2).glom().collect()

In [ ]:
rdd2.repartition(6).glom().collect()

In [ ]:
rdd2.map(lambda x: (x%6, x)).partitionBy(6).glom().collect()
# idem as rdd.map(lambda x: (x,x)).partitionBy(6, partitionFunc=(lambda x: x%6)).glom().collect()

# Persistance

In [ ]:
rdd3 = sc.parallelize([random.randint(0,10) for _ in range(50)])\
    .map(lambda x: (x%5, x)).cache() #.checkpoint()

In [ ]:
grps = rdd3.groupByKey()

In [ ]:
grps.count()

In [ ]:
grps.mapValues(lambda l: [x for x in l]).collect()